In [1]:
import numpy as np
import pandas as pd

In [2]:
def gen_candidate_set(xis):
    return pd.DataFrame(np.array(np.meshgrid(*xis)).T.reshape(-1,len(xis)))*1.0

In [3]:
def gen_model_matrix(candidate, n):
    # only main model effects is implemented for the time being.
    # TODO: extend to more complicated model structures as well.
    return candidate.sample(n, replace=False)

In [4]:
xis = ([-1, 0, 1],
       [-1, 0, 1],
       [-1, 0, 1],
       [-1, 0, 1],
       [-1, 0, 1],
       [-1, 0, 1],
       [-1, 0, 1],
       [-1, 0, 1])

# TODO: put this in a try catch to make sure that inv_mat exists
candidate = gen_candidate_set(xis)
X = gen_model_matrix(candidate, 12)
candidate.drop(X.index, inplace=True)
X

,0,1,2,3,4,5,6,7
1297,-1.0,0.0,-1.0,-1.0,0.0,1.0,0.0,-1.0
909,-1.0,-1.0,1.0,-1.0,1.0,-1.0,0.0,-1.0
4242,0.0,-1.0,-1.0,0.0,0.0,1.0,1.0,0.0
1550,-1.0,1.0,0.0,-1.0,0.0,-1.0,1.0,-1.0
1344,0.0,-1.0,1.0,0.0,0.0,1.0,0.0,-1.0
6163,1.0,0.0,-1.0,-1.0,0.0,0.0,1.0,1.0
754,1.0,0.0,1.0,-1.0,-1.0,-1.0,0.0,-1.0
2675,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,0.0
5102,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0
4000,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0


In [5]:
def find_best_candidate(inverse_mat, data=candidate):

    # calculate variance of remaining experiments
    variance_experiment = np.array([])
    for i in range(data.shape[0]):
        add_experiment = data.iloc[[i]]
        variance_experiment = np.append(variance_experiment, (add_experiment @ inverse_mat @ add_experiment.T).iat[0,0])

    # Find best experiment to add to the design (from candidates)
    candidate = pd.DataFrame(np.hstack((data, np.array(variance_experiment).reshape(-1,1))))
    max_var = np.argmax(candidate.iloc[:, -1])
    varmax_candidate = candidate.iloc[[max_var], :-1]

    return varmax_candidate

In [6]:
def find_worst_experiment(inverse_mat, data=X):
    # calculate variance of experiments already in the design
    variance_design = np.array([])
    for i in range(data.shape[0]):
        rem_experiment = data.iloc[[i]]
        variance_design = np.append(variance_design, (rem_experiment @ inverse_mat @ rem_experiment.T).iat[0,0])

    # Find worst experiment to remove from the design (from design)
    X = pd.DataFrame(np.hstack((data, np.array(variance_design).reshape(-1,1))))
    min_var = np.argmin(X.iloc[:, -1])
    varmin_experiment = X.iloc[[min_var], :-1]

    return varmin_experiment

In [7]:
def calculate_inv_mat(df=X):
    inf_mat = df.T @ df # calculate information matrix
    return np.linalg.inv(inf_mat) # calculate inverse of information matrix

In [8]:
def add_best_experiment(can=candidate, df=X):
    varmax = find_best_candidate(inverse_mat=calculate_inv_mat(df), data=can)
    # add the best candidate experiment
    df = df.append(varmax)
    # TODO: change the below to a function
    # drop the best candidate experiment from the candidate set
    drop_row = np.equal(can, np.asarray(varmax)).all(1) # find row where all elements are equal to the varmax
    can = can.drop(can.index[np.where(drop_row)[0]], inplace=False) # drop that row
    return df, can

In [9]:
# Something is off with the variable scopes here, need to investigate
# X_1, candidate_1 = add_best_experiment()
# X_1

In [10]:
def remove_worst_experiment(df=X):
    varmin = find_worst_experiment(inverse_mat=calculate_inv_mat(df), data=df)
    # drop the worst experiment from X
    drop_row = np.equal(df, np.asarray(varmin)).all(1)
    df = df.drop(df.index[np.where(drop_row)[0]], inplace=False)
    return df

In [11]:
# Something is off with the variable scopes here, need to investigate
# X_rem = remove_worst_experiment(df=X_1)
# X_rem

In [12]:
count = 0
delta = float('inf')
epsilon = 0.000000001
max_it = 100
det = np.array([0])

while (delta >= epsilon) & (count < max_it):
    if count%2==0:
        # add
        varmax = find_best_candidate(data=X, inverse_mat=calculate_inv_mat(X))
        # add the best candidate experiment
        X = X.append(varmax)
        # drop the best candidate experiment from the candidate set
        drop_row = np.equal(candidate, np.asarray(varmax)).all(1) # find row where all elements are equal to the varmax
        candidate.drop(candidate.index[np.where(drop_row)[0]], inplace=True) # drop that row
    else:
        # remove
        varmin = find_worst_experiment(data=X, inverse_mat=calculate_inv_mat(X))
        # drop the worst experiment from X
        drop_row = np.equal(X, np.asarray(varmin)).all(1)
        X.drop(X.index[np.where(drop_row)[0]], inplace=True)
    print(count, X.shape)
    det_end = np.linalg.det(X.T @ X)
    # print(count, det_end)
    delta = abs(X.shape[1]**X.shape[1]-det_end)
    # print(count, delta)
    np.append(det, det_end)
    count+=1

0 (13, 8)
1 (12, 8)
2 (13, 8)
3 (12, 8)
4 (13, 8)
5 (11, 8)
6 (12, 8)
7 (10, 8)
8 (11, 8)
9 (9, 8)
10 (10, 8)
11 (8, 8)
12 (9, 8)
13 (7, 8)
14 (8, 8)
15 (7, 8)


LinAlgError: Singular matrix

In [ ]:
X

In [ ]:
X.T @ X